# 기말과제: 인공지능적 영상 인식 기법 개발 

## 과제 개요: Image Classification on CIFAR-10 DB


### 과제 목표
본 기말 과제에서는 10개 class에 대해 32x32x3 차원의 영상 총 60,000장으로 구성되는 CIFAR-10 DB (https://www.cs.toronto.edu/~kriz/cifar.html)의 분류 성능을 극대화하는 영상 분류 기법을 구현하는 것을 목표로 한다.

### 과제 운영 방식
학생 개개인의 프로그래밍 실력 및 중점적으로 학습하고 싶은 세부적인 부분에 따라 아래의 두 가지 유형 중 하나를 선택하여 수행한다.
* 유형1: classifier를 통일한 상태에서 해당 classifier의 입력으로 들어오는 data feature vector를 생성하는 기법을 자체적으로 구현함. 외부 코드 차용은 원칙적으로 불가하며, 최종적인 판별 결과보다 개발 과정에서 수행한 다양한 시도와 실험을 기준으로 평가함.
* 유형2: 모든 외부 코드의 차용을 원칙적으로 허용하되, 활용하는 외부 코드에 대해 line-by-line으로 주석 설명문을 작성하거나 코드를 수정/개선하는 등 스스로 상세하게 이해하여 활용함. 최종적인 판별 결과를 극대화하는 것을 목표로 하며 최종 정확도를 기준으로 평가함.

이때 유형 1, 유형 2 모두 아래의 기본적인 외부 패키지는 활용이 허용된다.
* Numpy
* Pandas
* Scikit-image (skimage.feature module은 사용 불가)
* OpenCV (features2d, objdetect, dnn, ml, 및 extra modules는 사용 불가)

### 과제 제출물
별도 표기된 항목을 제외하고는 아래 항목을 과제 종료시(16주차 수업시간 12/20일 13시까지) 제출한다. 이때 파일 형식 및 파일명 등의 규정은 가상대학 과제 출제 게시글을 참조한다.
* 과제 계획서: 
 - 어떤 유형의 과제를 수행할지, 어떤 방법론을 활용할지 조사하고 계획을 문서로 정리함 
 - 12주차 수업시간(11/22일 13시)까지, pdf 형식, font 크기 11, 3페이지 이내
 - 파일명 "공학설계\_기말과제-계획서\_학번.pdf" 예: 공학설계\_기말과제-계획서\_2018_1234.pdf
* 과제 보고서: 
 - 완료된 과제의 구현된/활용된 기법의 이론적인 개요, 
 - 구현한/활용한 코드의 클래스/함수 등에 대해 API reference 형식 정리 (주요 알고리즘은 해당 함수 설명 내에 상세 내용 정리), 
 - [정량적 실험 결과]-구현된 기법(들)의 테스트셋 정확도 수치(들 및 비교표), 
 - [정성적 실험 결과]-구현된 기법에서 분류가 성공/실패하는 영상의 예시 및 원인에 대한 논의, 참고문헌 등을 포함함 
 - pdf 형식, font 크기 10, 8페이지 이내
 - 파일명 "공학설계\_기말과제-보고서\_학번.pdf" 예: 공학설계\_기말과제-보고서\_2018_1234.pdf
* 과제 발표 동영상: 
 - 과제 보고서의 내용을 요약 정리한 프리젠테이션 동영상 
 - mp4/mov/avi 등 형식, 3분 이내
 - 파일명 "공학설계\_기말과제-발표\_학번.pdf" 예: 공학설계\_기말과제-발표\_2018_1234.pdf 
* 과제 코드: 
 - CIFAR-10 DB 폴더를 제외한 프로젝트 코드를 모두 압축한 파일 
 - zip 형식, 단일 파일, 
 - 파일명 "공학설계\_기말과제-코드\_학번.pdf" 예: 공학설계\_기말과제-코드\_2018_1234.pdf

### CIFAR-10 DB Loading

아래 CELL 1-2은 CIFAR DB를 다운로드 받고 본 프로그램에서 활용할 수 있도록 읽어오는 코드를 제시한다. 
코드를 실행시키면 DB를 다운로드 받아서 data와 참값 class 번호를 각각 변수 X와 y에 저장한다.
이때 학습용 데이터와 테스트용 데이터를 각각 X_train 및 y_test과 X_test 및 y_test의 변수로 분할한다.

** 혹시 아래 shell 스크립트가 실행되지 않을 경우 가상대학에 업로드한 데이터 파일을 다운받고 아래의 코드와 호환되도록 경로를 지정하시오. **

In [1]:
### CELL-1 ###
# Download the CIFAR-10 DB to the current folder
!bash get_datasets.sh

'bash'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
### CELL-2 ###
# Load the raw CIFAR-10 data
from data_utils import load_CIFAR10

cifar10_dir = 'cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


## 유형 1: Fixed classifier type (Linear SVM)

feature 생성 함수를 별도의 셀 또는 별도의 .py 파일에 구현하고 아래 셀에서 호출하여 f_train 변수로 저장하고, 이를 이용하여 linear SVM을 학습하시오.
이때 다양한 알고리즘을 실험하고 최고의 정확도 결과를 도출하는 알고리즘을 개발하시오.

In [3]:
### Type-1 ###
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
clf = svm.SVC(kernel='linear')

# YOUR FEATURE GENERATION CODE HERE #
f_train = generate_features(X_train) # placeholder code, 자체 feature 생성 함수 호출 등으로 대체함
# ...
f_test = generate_features(X_test) # placeholder code, 자체 feature 생성 함수 호출 등으로 대체함
# END YOUR CODE #


### APPLY Linear SVM CLASSIFIER - 수정 가능하다고 명시한 코드 외에는 고정함 ###
# import linear SVM class from scikit-learn
from sklearn.svm import LinearSVC

# initialize linear SVM class object - LinearSVC 학습의 옵션 설정용 argument는 자유롭게 변경 가능함
my_linsvm_cls = LinearSVC(max_iter=10000)

# train linear SVM class object from X_train and y_train
my_linsvm_cls.fit(f_train, y_train)

# compute classification results for X_test and compute classification score by comparing with y_test
acc = my_linsvm_cls.score(f_test,y_test)

# check results - 이 부분 코드도 보고서에 작성하고자 하는 내용에 따라 자유롭게 수정함
print(acc)
# more code...
# 

ModuleNotFoundError: No module named 'sklearn'

## 유형 2: Open Source type

본 과제의 문제에 대한 기법을 조사하고, 성능이 좋은 기법을 찾아서 코드를 읽고 이해하고 주어진 데이터에 대해 실행하여 결과를 도출하시오.

In [ ]:
### Type-2 ###

# YOUR OPEN SOURCE CODE HERE #

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
# Currently, memory growth needs to be the same across GPUs 
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True) 
            logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    except RuntimeError as e: 
        # Memory growth must be set before GPUs have been initialized
        print(e)



In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import platform
print(platform.python_version())

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
pip list